In [1]:
# ▶️ 1. 로컬 디렉토리 생성
import os
folder_path = './my_project'
os.makedirs(folder_path, exist_ok=True)

In [12]:
# ▶️ 2. 필요한 모듈
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import pandas as pd
import time

In [13]:
# ▶️ 3. API 키 설정
tmdb_api_key = 'efc7d809310cb48568b7916e04369dbe'
omdb_api_key = 'd503f224'

In [14]:
# ▶️ 4. API URL 설정
tmdb_discover_url = 'https://api.themoviedb.org/3/discover/movie'
tmdb_detail_url = 'https://api.themoviedb.org/3/movie/{}'
tmdb_keywords_url = 'https://api.themoviedb.org/3/movie/{}/keywords'
omdb_url = 'http://www.omdbapi.com/'

headers = {'Accept': 'application/json'}
language = 'ko-KR'

In [15]:
# ▶️ 5. 전체 데이터 저장 리스트
all_movies = []

In [16]:
# 🔁 재시도 가능한 세션
def requests_retry_session(retries=3, backoff_factor=1.0, status_forcelist=(500, 502, 504)):
    session = requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('https://', adapter)
    session.mount('http://', adapter)
    return session

# 🎬 연도별 영화 수집
all_movies = []

for year in range(2015, 2026):
    year_movies = []
    page = 1
    total_pages = 1

    while page <= total_pages:
        params = {
            'api_key': tmdb_api_key,
            'language': language,
            'sort_by': 'popularity.desc',
            'primary_release_year': year,
            'page': page
        }

        try:
            response = requests_retry_session().get(tmdb_discover_url, headers=headers, params=params, timeout=10)
            data = response.json()
        except Exception as e:
            print(f"❌ TMDB 요청 실패 (연도: {year}, 페이지: {page}) → {e}")
            page += 1
            continue

        if page == 1:
            total_pages = data.get('total_pages', 1)

        for movie in data.get('results', []):
            tmdb_id = movie.get('id')
            title = movie.get('title')
            release_date = movie.get('release_date')
            media_type = "movie"
            vote_average = movie.get('vote_average')

            try:
                detail_resp = requests_retry_session().get(tmdb_detail_url.format(tmdb_id), headers=headers,
                                                           params={'api_key': tmdb_api_key, 'language': language}, timeout=10)
                detail = detail_resp.json()
            except Exception as e:
                print(f"⚠️ TMDB 상세 실패 - {tmdb_id} : {e}")
                continue

            imdb_id = detail.get('imdb_id')
            runtime = detail.get('runtime')
            genres = ", ".join([g['name'] for g in detail.get('genres', [])])
            production = detail.get('production_companies', [])
            production_company = production[0]['name'] if production else None

            try:
                keywords_resp = requests_retry_session().get(tmdb_keywords_url.format(tmdb_id), headers=headers,
                                                             params={'api_key': tmdb_api_key}, timeout=10)
                keywords_data = keywords_resp.json()
                tags = ", ".join([kw['name'] for kw in keywords_data.get('keywords', [])])
            except:
                tags = None

            director = cast = country = language_omdb = plot = imdb_rating = imdb_votes = None
            if imdb_id:
                try:
                    omdb_params = {'apikey': omdb_api_key, 'i': imdb_id}
                    omdb_resp = requests_retry_session().get(omdb_url, params=omdb_params, timeout=10)
                    omdb_data = omdb_resp.json()

                    director = omdb_data.get('Director')
                    cast = omdb_data.get('Actors')
                    country = omdb_data.get('Country')
                    language_omdb = omdb_data.get('Language')
                    plot = omdb_data.get('Plot')
                    imdb_rating = omdb_data.get('imdbRating')
                    imdb_votes = omdb_data.get('imdbVotes')
                except:
                    print(f"⚠️ OMDb 요청 실패 - imdb_id: {imdb_id}")
                    continue

            movie_data = {
                'title': title,
                'release_date': release_date,
                'type': media_type,
                'runtime': runtime,
                'genre': genres,
                'director': director,
                'cast': cast,
                'country': country,
                'language': language_omdb,
                'production_company': production_company,
                'synopsis': plot,
                'tags': tags,
                'imdb_rating': imdb_rating,
                'imdb_review_count': imdb_votes,
                'imdb_id': imdb_id
            }

            year_movies.append(movie_data)
            time.sleep(0.3)

        print(f"[{year}] 페이지 {page}/{total_pages} 수집 완료")
        page += 1

    # 📝 체크포인트 저장
    if year_movies:
        year_df = pd.DataFrame(year_movies)
        year_df.to_csv(f'{folder_path}/tmdb_imdb_checkpoint_{year}.csv', index=False)
        all_movies.extend(year_movies)
        print(f"✅ 체크포인트 저장: {year}")
    else:
        print(f"⚠️ 연도 {year}에 저장할 데이터가 없습니다.")

[2015] 페이지 1/1258 수집 완료
[2015] 페이지 2/1258 수집 완료
[2015] 페이지 3/1258 수집 완료
[2015] 페이지 4/1258 수집 완료
[2015] 페이지 5/1258 수집 완료
[2015] 페이지 6/1258 수집 완료
[2015] 페이지 7/1258 수집 완료
[2015] 페이지 8/1258 수집 완료
[2015] 페이지 9/1258 수집 완료
[2015] 페이지 10/1258 수집 완료
[2015] 페이지 11/1258 수집 완료
[2015] 페이지 12/1258 수집 완료
[2015] 페이지 13/1258 수집 완료
[2015] 페이지 14/1258 수집 완료
[2015] 페이지 15/1258 수집 완료
[2015] 페이지 16/1258 수집 완료
[2015] 페이지 17/1258 수집 완료
[2015] 페이지 18/1258 수집 완료
[2015] 페이지 19/1258 수집 완료
[2015] 페이지 20/1258 수집 완료
[2015] 페이지 21/1258 수집 완료
[2015] 페이지 22/1258 수집 완료
[2015] 페이지 23/1258 수집 완료
[2015] 페이지 24/1258 수집 완료
[2015] 페이지 25/1258 수집 완료
[2015] 페이지 26/1258 수집 완료
[2015] 페이지 27/1258 수집 완료
[2015] 페이지 28/1258 수집 완료
[2015] 페이지 29/1258 수집 완료
[2015] 페이지 30/1258 수집 완료
[2015] 페이지 31/1258 수집 완료
[2015] 페이지 32/1258 수집 완료
[2015] 페이지 33/1258 수집 완료
[2015] 페이지 34/1258 수집 완료
[2015] 페이지 35/1258 수집 완료
[2015] 페이지 36/1258 수집 완료
[2015] 페이지 37/1258 수집 완료
[2015] 페이지 38/1258 수집 완료
[2015] 페이지 39/1258 수집 완료
[2015] 페이지 40/1258 수집 완료
[2015] 페이

In [ ]:
# 📦 전체 저장
if all_movies:
    final_df = pd.DataFrame(all_movies)
    final_df.to_csv(f'{folder_path}/tmdb_imdb_combined_2015_2025.csv', index=False)
    print("✅ 전체 저장 완료!")
else:
    print("❌ 수집된 데이터가 없습니다.")

In [ ]:
# ▶️ 6. 연도별 수집 루프
for year in range(2015, 2026):
    year_movies = []  # 각 연도마다 리스트 초기화
    page = 1
    total_pages = 1

    while page <= total_pages:
        params = {
            'api_key': tmdb_api_key,
            'language': language,
            'sort_by': 'popularity.desc',
            'primary_release_year': year,
            'page': page
        }

        try:
            response = requests.get(tmdb_discover_url, headers=headers, params=params)
            data = response.json()
        except Exception as e:
            print(f"❌ TMDB 요청 실패 (연도: {year}, 페이지: {page}) → {e}")
            break

        if page == 1:
            total_pages = data.get('total_pages', 1)

        for movie in data.get('results', []):
            tmdb_id = movie.get('id')
            title = movie.get('title')
            release_date = movie.get('release_date')
            media_type = "movie"
            vote_average = movie.get('vote_average')

            try:
                detail_resp = requests.get(tmdb_detail_url.format(tmdb_id), headers=headers,
                                           params={'api_key': tmdb_api_key, 'language': language})
                detail = detail_resp.json()
            except Exception as e:
                print(f"⚠️ TMDB 상세 실패 - {tmdb_id} : {e}")
                continue

            imdb_id = detail.get('imdb_id')
            runtime = detail.get('runtime')
            genres = ", ".join([g['name'] for g in detail.get('genres', [])])
            production = detail.get('production_companies', [])
            production_company = production[0]['name'] if production else None

            try:
                keywords_resp = requests.get(tmdb_keywords_url.format(tmdb_id), headers=headers,
                                             params={'api_key': tmdb_api_key})
                keywords_data = keywords_resp.json()
                tags = ", ".join([kw['name'] for kw in keywords_data.get('keywords', [])])
            except:
                tags = None

            director = cast = country = language_omdb = plot = imdb_rating = imdb_votes = None
            if imdb_id:
                try:
                    omdb_params = {'apikey': omdb_api_key, 'i': imdb_id}
                    omdb_resp = requests.get(omdb_url, params=omdb_params)
                    omdb_data = omdb_resp.json()

                    director = omdb_data.get('Director')
                    cast = omdb_data.get('Actors')
                    country = omdb_data.get('Country')
                    language_omdb = omdb_data.get('Language')
                    plot = omdb_data.get('Plot')
                    imdb_rating = omdb_data.get('imdbRating')
                    imdb_votes = omdb_data.get('imdbVotes')
                except:
                    print(f"⚠️ OMDb 요청 실패 - imdb_id: {imdb_id}")
                    continue

            movie_data = {
                'title': title,
                'release_date': release_date,
                'type': media_type,
                'runtime': runtime,
                'genre': genres,
                'director': director,
                'cast': cast,
                'country': country,
                'language': language_omdb,
                'production_company': production_company,
                'synopsis': plot,
                'tags': tags,
                'imdb_rating': imdb_rating,
                'imdb_review_count': imdb_votes,
                'imdb_id': imdb_id
            }

            year_movies.append(movie_data)
            time.sleep(0.3)  # 과도한 요청 방지

        print(f"[{year}] 페이지 {page}/{total_pages} 수집 완료")
        page += 1

    # ▶️ 연도별 임시 저장 (체크포인트)
    if year_movies:
        year_df = pd.DataFrame(year_movies)
        year_df.to_csv(f'{folder_path}/tmdb_imdb_checkpoint_{year}.csv', index=False)
        all_movies.extend(year_movies)
        print(f"✅ 체크포인트 저장: {year}")
    else:
        print(f"⚠️ 연도 {year}에 저장할 데이터가 없습니다.")

# ▶️ 7. 전체 데이터 저장
if all_movies:
    final_df = pd.DataFrame(all_movies)
    final_df.to_csv(f'{folder_path}/tmdb_imdb_combined_2015_2025.csv', index=False)
    print("✅ 전체 저장 완료!")
else:
    print("❌ 수집된 데이터가 없습니다.")